In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

In [ ]:
# Fig 9(a)
# Step 1: Download the data at this https://snap.stanford.edu/data/feather-lastfm-social.html
# Step 2: Replace with your data file path below
file_path = '/home/iisertvm/Desktop/Network_data/lasftm_asia/lastfm_asia_edges.csv'
G = nx.read_edgelist(file_path, delimiter=',')


def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
        # else:
            #print("node", node, "deosn't fails", "# neigbors fails = ", count)
    #else:
        # print("node itself doesn't fail")
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        degrees = degrees.astype(float)
        avg_degree = np.mean(degrees)
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict

def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
#         print(H.degree())
#         print(H.degree(i), avg_color_degrees_dict[color_dict[i]])
        # print(H.degree(i))
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    count = 1
    not_failed_nodes =[]
    failed_nodes = []
 
    while count != len(not_failed_nodes):
        #print("entered while")
        count = 0
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
#                     if 1-prot_prob < random.uniform(0,1):   old code
#                     if prot_prob < random.uniform(0,1):    # new code

                        
                    #print("entered protection loop: node removed =", each)
                    #print(each, "immune node removed")
#                         H.remove_node(each)
#                         failed_nodes.append(each)
#                     current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)

        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or (each in immune_nodes and prot_prob==1.0) or (each in immune_nodes and prot_prob==0.7) or (each in immune_nodes and prot_prob==0.4):
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
        else:
            #print("continue started")
            current_failed_nodes = []
            not_failed_nodes = []
            not_failed_nodes_desc ={}
            continue
    #print("failed nodes = ", sorted(failed_nodes))
       
    return failed_nodes


# G =  nx.read_edgelist('/home/iisertvm/Desktop/Mitigation python/facebook_combined.txt', create_using=nx.Graph(), nodetype=int)
file_path = '/home/iisertvm/Desktop/Network_data/lasftm_asia/lastfm_asia_edges.csv'
G = nx.read_edgelist(file_path, delimiter=',')
prot_prob = [1.0,0.7,0.4,0.0]
# prot_prob = [1.0]

Phi = np.arange(0.80,0.90,0.005)
N_nodes = len(G.nodes)


realizations = 150
immune_node_size =[] 
selected_node_size =[]

for protection_prob in prot_prob:
    suriviving_prob_array = []
    for thetha in Phi:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean([int(x) for x in degrees])


            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes

            avg_degree_selected_nodes = np.mean([H.degree(i) for i in selected_nodes])

            color_dict, avg_color_degrees_dict = avg_color_degree(H)
            # print(min(avg_color_degrees_dict))

            immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
            # print(len(immune_nodes))
#             print(" theta =", thetha," ", "# selected nodes =", len(selected_nodes), " ", "immune nodes =", len(immune_nodes), " ", "avg. degree of selected nodes =", avg_degree_selected_nodes)

#             failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)

#             suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade) )/ len(I.nodes)
#             intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
#         suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
#     print(suriviving_prob_array)
#     plt.plot(Phi, suriviving_prob_array, label = protection_prob)

# plt.plot(Phi,immune_node_size,'--', label = "immune" )
# plt.plot(Phi,selected_node_size,':', label = "Selected" )
print(selected_node_size)

print(immune_node_size)
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.show()



In [ ]:
# Fig 9(b)
def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
        # else:
            #print("node", node, "doesn't fails", "# neigbors fails = ", count)
    #else:
        # print("node itself doesn't fail")
    return selected_nodes


def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(G.degree())
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean(degrees)
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict

def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    protected_nodes_list = []
    for i in failed_nodes:
#         print(H.degree())
#         print(H.degree(i), avg_color_degrees_dict[color_dict[i]])
        if H.degree(i) > min_value:
            protected_nodes_list.append(i)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
#                     if 1-prot_prob < random.uniform(0,1):   old code
#                     if prot_prob < random.uniform(0,1):    # new code

                        
                    #print("entered protection loop: node removed =", each)
                    #print(each, "immune node removed")
#                         H.remove_node(each)
#                         failed_nodes.append(each)
#                     current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
#         else:
#             #print("continue started")
#             current_failed_nodes = []
#             not_failed_nodes = []
#             not_failed_nodes_desc ={}
#             continue
    #print("failed nodes = ", sorted(failed_nodes))
       
    return failed_nodes


degree_sequence_erdos = list(G.degree())
degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
average_degree_of_original_graph = np.round(np.mean([int(x) for x in degrees]),2)

Phi = [0.75,0.80,0.825,0.85]
prot_prob =np.arange(0.0,1.1,0.1)


realizations =150


immune_node_size =[]
selected_node_size =[]
#suriviving_prob_array = []

for thetha in Phi:

   
    suriviving_prob_array = []
   
    for protection_prob in prot_prob:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):

            # Get the list of the degrees
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean(degrees)
            #print ("average degree of original graph = ",avg_degree)
            # print("Original graph Degree")
            # print(G.degree())
            # nx.draw(G, with_labels =1)
            # plt.show()

            #removing nodes having degree = 0
            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#             print(" Selected nodes = ", selected_nodes)



#             avg_degree_of_subgraphs = avg_degree_of_subgraph(H,selected_nodes)
            color_dict, avg_color_degrees_dict = avg_color_degree(H)
#             print("avg_degree_of_subgraphs: ", avg_degree_of_subgraphs)

            #print("average degree of all subgraphs = ", avg_degree_of_subgraphs)

#             immune_nodes = protected_nodes(avg_degree_of_subgraphs,selected_nodes)
            immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
#             print("Protected set = ", immune_nodes)
            #################################################################################################################
            failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)
            #print("failed_nodes_at_the_end_cascade = ", failed_nodes_at_the_end_cascade)
            suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade) )/ len(I.nodes)
           
            intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
        suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
       
#     print(len(Phi), len(suriviving_prob_array))
    print(suriviving_prob_array)

    plt.plot(prot_prob, suriviving_prob_array, label = thetha)
       
# plt.plot(Phi,immune_node_size,'--', label = "immune nodes" )
# plt.plot(Phi,selected_node_size,':', label = "Selected nodes" )
# print(immune_node_size)
plt.xlabel("Probability of Immunization")
plt.ylabel("System's surviving probability")
plt.legend(loc ="best")
plt.show()


In [ ]:
# Fig 9(c)
# Step 1: Download the data at this https://snap.stanford.edu/data/ca-CondMat.html
# Step 2: Replace with your data file path below
file_path='/home/iisertvm/Desktop/Network_data/CA-CondMat.txt'
G =  nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)

## last run 26 july 2024

###################### ER coloring algorith #########################
def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -2) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 2) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
        # else:
            #print("node", node, "deosn't fails", "# neigbors fails = ", count)
    #else:
        # print("node itself doesn't fail")
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean(degrees)
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict

def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
#         print(H.degree())
#         print(H.degree(i), avg_color_degrees_dict[color_dict[i]])
        # print(H.degree(i))
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    count = 1
    not_failed_nodes =[]
    failed_nodes = []
 
    while count != len(not_failed_nodes):
        #print("entered while")
        count = 0
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
#                     if 1-prot_prob < random.uniform(0,1):   old code
#                     if prot_prob < random.uniform(0,1):    # new code

                        
                    #print("entered protection loop: node removed =", each)
                    #print(each, "immune node removed")
#                         H.remove_node(each)
#                         failed_nodes.append(each)
#                     current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)

        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or (each in immune_nodes and prot_prob==1.0) or (each in immune_nodes and prot_prob==0.7) or (each in immune_nodes and prot_prob==0.4):
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
        else:
            #print("continue started")
            current_failed_nodes = []
            not_failed_nodes = []
            not_failed_nodes_desc ={}
            continue
    #print("failed nodes = ", sorted(failed_nodes))
       
    return failed_nodes


# G =  nx.read_edgelist('/home/iisertvm/Desktop/Mitigation python/facebook_combined.txt', create_using=nx.Graph(), nodetype=int)
G =  nx.read_edgelist('/home/iisertvm/Desktop/Network_data/CA-CondMat.txt', create_using=nx.Graph(), nodetype=int)
prot_prob = [1.0,0.7,0.4,0.0]
# prot_prob = [1.0]

Phi = np.arange(0.80,0.90,0.005)
N_nodes = len(G.nodes)
degree_sequence_erdos = list(G.degree())
degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
average_degree_of_original_graph = np.mean(degrees)

realizations = 150
immune_node_size =[] 
selected_node_size =[]

for protection_prob in prot_prob:
    suriviving_prob_array = []
    for thetha in Phi:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean([int(x) for x in degrees])


            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes

            avg_degree_selected_nodes = np.mean([H.degree(i) for i in selected_nodes])

            color_dict, avg_color_degrees_dict = avg_color_degree(H)
            # print(min(avg_color_degrees_dict))

            immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
            # print(len(immune_nodes))
#             print(" theta =", thetha," ", "# selected nodes =", len(selected_nodes), " ", "immune nodes =", len(immune_nodes), " ", "avg. degree of selected nodes =", avg_degree_selected_nodes)

            failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)

            suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade) )/ len(I.nodes)
            intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
        suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
    print(suriviving_prob_array)
    plt.plot(Phi, suriviving_prob_array, label = protection_prob)

plt.plot(Phi,immune_node_size,'--', label = "immune" )
# plt.plot(Phi,selected_node_size,':', label = "Selected" )
print(immune_node_size)
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.show()



In [ ]:
# Fig 9(d)
# last run 20/02/24
###################### ER coloring algorith (systems's prob. vs Immunization prob.) #########################
def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -2) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 2) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
        # else:
            #print("node", node, "doesn't fails", "# neigbors fails = ", count)
    #else:
        # print("node itself doesn't fail")
    return selected_nodes




def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(G.degree())
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean(degrees)
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict

def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    protected_nodes_list = []
    for i in failed_nodes:
#         print(H.degree())
#         print(H.degree(i), avg_color_degrees_dict[color_dict[i]])
        if H.degree(i) > min_value:
            protected_nodes_list.append(i)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
#                     if 1-prot_prob < random.uniform(0,1):   old code
#                     if prot_prob < random.uniform(0,1):    # new code

                        
                    #print("entered protection loop: node removed =", each)
                    #print(each, "immune node removed")
#                         H.remove_node(each)
#                         failed_nodes.append(each)
#                     current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
#         else:
#             #print("continue started")
#             current_failed_nodes = []
#             not_failed_nodes = []
#             not_failed_nodes_desc ={}
#             continue
    #print("failed nodes = ", sorted(failed_nodes))
       
    return failed_nodes

prot_prob =np.arange(0.0,1.1,0.1)

Phi = [0.80,0.825,0.85,0.875]

realizations =50
# average_degree_of_original_graph = (p*N_nodes)  #for probability model erdos-renyi
# average_degree_of_original_graph = (2*m) /N_nodes    #for edge model erdos-renyi
immune_node_size =[]
selected_node_size =[]
#suriviving_prob_array = []
average_degree_of_original_graph = 8.08
for thetha in Phi:

   
    suriviving_prob_array = []
   
    for protection_prob in prot_prob:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):

        ###############################################################################################################
        #Generate the graph
           
            # Erdos-Renyi graph N- nodes and p - probability of joining randomly selected two nodes
#             G = nx.erdos_renyi_graph(N_nodes,p)
            

            # nx.draw(G, with_labels = True)
            # plt.show()

            # Get the list of the degrees
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean(degrees)
            #print ("average degree of original graph = ",avg_degree)
            # print("Original graph Degree")
            # print(G.degree())
            # nx.draw(G, with_labels =1)
            # plt.show()

            #removing nodes having degree = 0
            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#             print(" Selected nodes = ", selected_nodes)



#             avg_degree_of_subgraphs = avg_degree_of_subgraph(H,selected_nodes)
            color_dict, avg_color_degrees_dict = avg_color_degree(H)
#             print("avg_degree_of_subgraphs: ", avg_degree_of_subgraphs)

            #print("average degree of all subgraphs = ", avg_degree_of_subgraphs)

#             immune_nodes = protected_nodes(avg_degree_of_subgraphs,selected_nodes)
            immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
#             print("Protected set = ", immune_nodes)
            #################################################################################################################
            failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)
            #print("failed_nodes_at_the_end_cascade = ", failed_nodes_at_the_end_cascade)
            suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade) )/ len(I.nodes)
           
            intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
        suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
       
#     print(len(Phi), len(suriviving_prob_array))
    print(suriviving_prob_array)

    plt.plot(prot_prob, suriviving_prob_array, label = thetha)
       
# plt.plot(Phi,immune_node_size,'--', label = "immune nodes" )
# plt.plot(Phi,selected_node_size,':', label = "Selected nodes" )
# print(immune_node_size)
plt.xlabel("Probability of Immunization")
plt.ylabel("System's surviving probability")
plt.legend(loc ="best")
plt.show()


In [ ]:
# Fig 9(e)
# Step 1: Download the data at this https://networkrepository.com/opsahl-powergrid.php
# Step 2: Replace with your data file path below

file_path = '/home/iisertvm/Desktop/Network_data/opsahl-powergrid/powergrid.txt'  # Replace with your file path
G = nx.read_adjlist(file_path, nodetype=int)

def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
        # else:
            #print("node", node, "deosn't fails", "# neigbors fails = ", count)
    #else:
        # print("node itself doesn't fail")
    
    return None

def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(H.degree())      # in original code, it's "G.degree()"
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean([int(x) for x in degrees])
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict

def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    # print(min_value)
    protected_nodes_list = []
    for i in failed_nodes:
#         print(H.degree())
#         print(H.degree(i), avg_color_degrees_dict[color_dict[i]])
        # print(H.degree(i))
        if H.degree(i) >min_value:
            protected_nodes_list.append(i)
    # print(protected_nodes_list)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
#                     if 1-prot_prob < random.uniform(0,1):   old code
#                     if prot_prob < random.uniform(0,1):    # new code

                        
                    #print("entered protection loop: node removed =", each)
                    #print(each, "immune node removed")
#                         H.remove_node(each)
#                         failed_nodes.append(each)
#                     current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or (each in immune_nodes and prot_prob==1.0) or (each in immune_nodes and prot_prob==0.7) or (each in immune_nodes and prot_prob==0.4):
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
#         else:
#             #print("continue started")
#             current_failed_nodes = []
#             not_failed_nodes = []
#             not_failed_nodes_desc ={}
#             continue
    #print("failed nodes = ", sorted(failed_nodes))
       
    return failed_nodes


# file_path = '/home/iisertvm/Desktop/Network_data/opsahl-powergrid/powergrid.txt'  # Replace with your file path
# G = nx.read_adjlist(file_path, nodetype=int)

prot_prob = [1.0,0.7,0.4,0.0]
# prot_prob = [1.0]

Phi = np.arange(0.75,0.90,0.005)
N_nodes = len(G.nodes)
degree_sequence_erdos = list(G.degree())
degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
average_degree_of_original_graph = np.round(np.mean([int(x) for x in degrees]),2)

realizations = 150
immune_node_size =[] 
selected_node_size =[]


for protection_prob in prot_prob:
    suriviving_prob_array = []
    for thetha in Phi:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
#         potent_nodes_dict = {}
#         potent_nodes_set = set()
        for i in range(realizations):
            degree_sequence_erdos = list(G.degreedos)[:,1]  #array of degrees
            avg_degree = np.mean([int(x) for x in degrees])


            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes

            avg_degree_selected_nodes = np.mean([H.degree(i) for i in selected_nodes])

            color_dict, avg_color_degrees_dict = avg_color_degree(H)
            # print(min(avg_color_degrees_dict))

            immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
#             for i in immune_nodes:
#                 potent_nodes_set.add(i)
#             potent_nodes_dict[thetha] = potent_nodes_set
            
            
            # print(len(immune_nodes))
#             print(" theta =", thetha," ", "# selected nodes =", len(selected_nodes), " ", "immune nodes =", len(immune_nodes), " ", "avg. degree of selected nodes =", avg_degree_selected_nodes)

#             failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)

#             suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade) )/ len(I.nodes)
#             intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
# #         print("potent_nodes = ", potent_nodes_dict)
#         suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
#     print(suriviving_prob_array)
#     plt.plot(Phi, suriviving_prob_array, label = protection_prob)

# plt.plot(Phi,immune_node_size,'--', label = "immune" )
# plt.plot(Phi,selected_node_size,':', label = "Selected" )
print(selected_node_size)
print(immune_node_size)
plt.xlabel("Threshold")
plt.ylabel("System's surviving probability")
plt.legend(loc ="upper right")
plt.title("Power Grid  |   "+"N = "+str(N_nodes)+" , " "<k> = "+str(average_degree_of_original_graph))
plt.show()



In [ ]:
# Fig 9(f)
def selecting_nodes(node,neighbor_list, selected_nodes,thetha):
 ################## Returns the list of failed nodes (Algorithm for selecting nodes)################
    frac_threshold = (H.degree(node) -1) /G.degree(node)
    if frac_threshold < thetha:
        count = 0
        for i in range(len(neighbor_list)):
            frac_threshold_neighbor = (H.degree(neighbor_list[i]) - 1) / H.degree(neighbor_list[i])
            if frac_threshold_neighbor < thetha:
                count += 1
        if count >= 2:
            #print("node", node, "fails", "# neigbors fails = ", count)
            selected_nodes.append(node)
        # else:
            #print("node", node, "doesn't fails", "# neigbors fails = ", count)
    #else:
        # print("node itself doesn't fail")
    return selected_nodes


def avg_color_degree(H):
    colors_dict = nx.coloring.greedy_color(H)
    degree_sequence_erdos = list(G.degree())
    max_color = max(list(colors_dict.values()))
    avg_color_dict = {}
    for i in range(max_color+1):
        avg = [x for x in colors_dict if colors_dict[x] == i]
        degree_sequence_erdos = list(H.degree(avg))
        degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
        avg_degree = np.mean(degrees)
        avg_color_dict[i] = avg_degree
    #print(colors_dict[2],avg_color_dict[colors_dict[2]])
    return colors_dict,avg_color_dict

def protected_nodes(color_dict, avg_color_degrees_dict,failed_nodes):
################ Returns the list of nodes whose degree is greater than the average degree of subgraphs ######
    min_value = min(avg_color_degrees_dict.values())
    protected_nodes_list = []
    for i in failed_nodes:
#         print(H.degree())
#         print(H.degree(i), avg_color_degrees_dict[color_dict[i]])
        if H.degree(i) > min_value:
            protected_nodes_list.append(i)
    return protected_nodes_list


def cascading_failure(G,H,immune_nodes,thetha, prot_prob):
################################ Cascading failure ######################################
    #print(H.degree())
    #initial_edge = random.choice(list(H.edges))
   
    initial_node = random.choice([i for i in list(H.nodes) if i not in immune_nodes ])
   
    #print("edge removed = ", initial_edge)
    #H.remove_edge(*initial_edge)
    H.remove_node(initial_node)         #randomly removing a node, avoiding the immume_nodes, to initiate the cascade
#     print("node removed =",initial_node)
    #print(H.degree())

    
    not_failed_nodes =[]
    failed_nodes = []
 
    while True:
        current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
        #print("current_failed_nodes = ", current_failed_nodes)
        #print("current_failed_nodes degree =", H.degree())
        for each in current_failed_nodes:
            #print("entered for loop: current_failed_nodes")
            if each in immune_nodes:
                if prot_prob ==1.0:
                    pass
                elif prot_prob ==0.0:
                    H.remove_node(each)
                    if each not in failed_nodes:
                        failed_nodes.append(each)
                else:
                    r = np.random.random(10000)
                    event = (r<prot_prob)
                    ratio = np.mean(event)
                    if np.random.choice(event) == False:
                        H.remove_node(each)
                        if each not in failed_nodes:
                            failed_nodes.append(each)
#                     if 1-prot_prob < random.uniform(0,1):   old code
#                     if prot_prob < random.uniform(0,1):    # new code

                        
                    #print("entered protection loop: node removed =", each)
                    #print(each, "immune node removed")
#                         H.remove_node(each)
#                         failed_nodes.append(each)
#                     current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]
            else :
                #print("entered else remove loop: node removed = ", each)
                #print(each, "Non -immune node removed")
                H.remove_node(each)
                if each not in failed_nodes:
                    failed_nodes.append(each)
#                 current_failed_nodes = [node for node in H.nodes if H.degree(node)/G.degree(node) < thetha]

        not_failed_nodes = [H.degree(node)/G.degree(node)  for node in H.nodes]
        not_failed_nodes_desc = {node:H.degree(node)/G.degree(node)  for node in H.nodes}
        #print("not_failed_nodes = ",not_failed_nodes)
        #print("not_failed_nodes_desc = ",not_failed_nodes_desc)
        count = 0
        for each in not_failed_nodes_desc:
            if not_failed_nodes_desc[each] >= thetha or each in immune_nodes:
                #print("entered if condition: each>= phi or each in immune")
                count +=1
        if count == len(not_failed_nodes):
            #print("Break: count == len(not_failed_nodes)")
            break
#         else:
#             #print("continue started")
#             current_failed_nodes = []
#             not_failed_nodes = []
#             not_failed_nodes_desc ={}
#             continue
    #print("failed nodes = ", sorted(failed_nodes))
       
    return failed_nodes


file_path = '/home/iisertvm/Desktop/Network_data/opsahl-powergrid/powergrid.txt'  # Replace with your file path
G = nx.read_adjlist(file_path, nodetype=int)

degree_sequence_erdos = list(G.degree())
degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
average_degree_of_original_graph = np.round(np.mean([int(x) for x in degrees]),2)

Phi = [0.75,0.80,0.825,0.85]
prot_prob =np.arange(0.0,1.1,0.1)


realizations =150


immune_node_size =[]
selected_node_size =[]
#suriviving_prob_array = []

for thetha in Phi:

   
    suriviving_prob_array = []
   
    for protection_prob in prot_prob:
        intermediate_suriviving_prob_array = []
        intermediate_immune_node_size =[]
        intermediate_selected_node_size =[]
        for i in range(realizations):

            # Get the list of the degrees
            degree_sequence_erdos = list(G.degree())
            degrees = np.array(degree_sequence_erdos)[:,1]  #array of degrees
            avg_degree = np.mean(degrees)
            #print ("average degree of original graph = ",avg_degree)
            # print("Original graph Degree")
            # print(G.degree())
            # nx.draw(G, with_labels =1)
            # plt.show()

            #removing nodes having degree = 0
            remove = [node for node,degree in dict(G.degree()).items() if degree == 0]
            G.remove_nodes_from(remove)

            N_nodes_new = list(G.nodes)    #list of nodes with k>0


            H = G.copy()   # graph contains nodes that have k > 0
            I = G.copy()
            degree_sequence_erdos = list(H.degree())


            selected_nodes = []
            for node in H.nodes:
                neighbor_list = [n for n in H.neighbors(node)]
                selecting_nodes(node,neighbor_list,selected_nodes,thetha) # return list of selected list of nodes
#             print(" Selected nodes = ", selected_nodes)



#             avg_degree_of_subgraphs = avg_degree_of_subgraph(H,selected_nodes)
            color_dict, avg_color_degrees_dict = avg_color_degree(H)
#             print("avg_degree_of_subgraphs: ", avg_degree_of_subgraphs)

            #print("average degree of all subgraphs = ", avg_degree_of_subgraphs)

#             immune_nodes = protected_nodes(avg_degree_of_subgraphs,selected_nodes)
            immune_nodes = protected_nodes(color_dict, avg_color_degrees_dict,selected_nodes)
#             print("Protected set = ", immune_nodes)
            #################################################################################################################
            failed_nodes_at_the_end_cascade = cascading_failure(G,H,immune_nodes,thetha,protection_prob)
            #print("failed_nodes_at_the_end_cascade = ", failed_nodes_at_the_end_cascade)
            suriviving_prob = (len(I.nodes) - len(failed_nodes_at_the_end_cascade) )/ len(I.nodes)
           
            intermediate_suriviving_prob_array.append(suriviving_prob)
            if protection_prob ==1.0:
                intermediate_immune_node_size.append(len(immune_nodes)/N_nodes)
                intermediate_selected_node_size.append(len(selected_nodes)/N_nodes)
        suriviving_prob_array.append(round(np.mean(intermediate_suriviving_prob_array),2))
        if protection_prob ==1.0:
            immune_node_size.append(np.mean(intermediate_immune_node_size))
            selected_node_size.append(np.mean(intermediate_selected_node_size))
       
#     print(len(Phi), len(suriviving_prob_array))
    print(suriviving_prob_array)

    plt.plot(prot_prob, suriviving_prob_array, label = thetha)
       
# plt.plot(Phi,immune_node_size,'--', label = "immune nodes" )
# plt.plot(Phi,selected_node_size,':', label = "Selected nodes" )
# print(immune_node_size)
plt.xlabel("Probability of Immunization")
plt.ylabel("System's surviving probability")
plt.legend(loc ="best")
plt.title("Power Grid  "+"N = "+str(N_nodes)+" , " "<k> = "+str(average_degree_of_original_graph))
plt.show()
